In [1]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.tile_providers import get_provider, Vendors

from pyproj import Transformer
transformer = Transformer.from_crs(28992, 3857)

output_notebook()

Loading BokehJS ...

Specificatie van de HydroBase url

In [2]:
api = "https://www.hydrobase.nl/api"
#api = "http://127.0.0.1:7001"

Invoer:
- line:  een lijn gespecificeerd door start en eind-coordinaten in rijksdriehoekcoordinaten: [xstart, ystart, xeind, yeind]

In [3]:
parameters = {"line": [127673, 462572, 128068, 461619]}
#parameters = {"line": [230935, 516062, 204889, 517581]}
parameters = {"line": [127696, 462473, 127882, 462167]}
parameters = {"line": [131180, 453432, 131609, 453280]}

Plotten van de lijn

In [4]:
p = figure()
tile_provider = get_provider(Vendors.CARTODBPOSITRON)
p.add_tile(tile_provider)

line = parameters["line"]
xx, yy = transformer.transform(xx=(line[0], line[2]), yy=(line[1], line[3]))
p.line(x = xx,
       y = yy,
       color = "green")

show(p)

Downloaden van de lagen gesampled over de lijn
- dtm: het gefilterde AHN, alles wat hier niet maaiveld is is no-data en wordt niet getoond
- dsm: het ongefilterde AHN, representatief voor de hoogte van objecten óp het maaiveld wanneer deze er zijn

In [5]:
response = requests.post(url=f"{api}/digital-terrain-model", json=parameters, verify=False)
if response.status_code == 200:
    dtm = response.json()

response = requests.post(url=f"{api}/digital-surface-model", json=parameters, verify=False)
if response.status_code == 200:
    dsm = response.json()

response = requests.post(url=f"{api}/canal-cross-sections", json=parameters, verify=False)
if response.status_code == 200:
    canal_xs = response.json()

response = requests.post(url=f"{api}/target-waterlevel-winter", json=parameters, verify=False)
if response.status_code == 200:
    target_wl_win = response.json()

response = requests.post(url=f"{api}/target-waterlevel-summer", json=parameters, verify=False)
if response.status_code == 200:
    target_wl_sum = response.json()

Plotten van het resultaat

In [6]:

p = figure(x_axis_label="afstand [m]",
          y_axis_label="hoogte [m NAP]")
p.multi_line(xs = [[coord[0] for coord in i] for i in dtm],
             ys = [[coord[1] for coord in i] for i in dtm],
             color = "green",
             legend_label="dtm")

p.multi_line(xs = [[coord[0] for coord in i] for i in canal_xs],
             ys = [[coord[1] for coord in i] for i in canal_xs],
       color = "orange",
       legend_label="waterloopprofiel")

p.multi_line(xs = [[coord[0] for coord in i] for i in target_wl_sum],
             ys = [[coord[1] for coord in i] for i in target_wl_sum],
       color = "cyan",
       legend_label="zomerpeil")

p.multi_line(xs = [[coord[0] for coord in i] for i in target_wl_win],
             ys = [[coord[1] for coord in i] for i in target_wl_win],
       color = "darkblue",
       legend_label="winterpeil")

show(p)